In [1]:
# gym stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# import helpers
import numpy as np
import random
import os

# import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env

In [2]:
Discrete(3).sample()

1

In [3]:
Box(0,3,shape=(3,3)).sample()

array([[1.3720378 , 1.0108801 , 0.42045   ],
       [0.12890296, 0.9596579 , 1.4612517 ],
       [0.8304746 , 1.6407046 , 2.260023  ]], dtype=float32)

In [4]:
Tuple((Discrete(3),Box(0,3,shape=(3,3)))).sample()

(2,
 array([[0.507527  , 0.6750211 , 2.3143785 ],
        [0.15607339, 0.42753857, 1.0038663 ],
        [0.7174326 , 1.1526558 , 2.9562056 ]], dtype=float32))

In [5]:
Dict({'length':Discrete(3),'width':Discrete(2),'box':Box(0,100,shape=(3,3))}).sample()

OrderedDict([('box',
              array([[66.64223  , 89.85359  , 24.784348 ],
                     [78.77613  ,  0.5045652, 52.817875 ],
                     [52.89297  , 51.825577 ,  7.0877304]], dtype=float32)),
             ('length', 2),
             ('width', 0)])

In [6]:
MultiBinary(8).sample()

array([1, 0, 0, 1, 0, 1, 1, 1], dtype=int8)

In [7]:
inputaction = MultiDiscrete([4,5]).sample()
print(inputaction)
print(inputaction[1])

[2 3]
3


In [8]:
Dict({'Reblaths':MultiDiscrete([41,41,41,41]),'Failstacks':MultiDiscrete([401,401,401,401,401])}).sample()['Reblaths'][0]

0

In [9]:
{'Reblaths':[41,41,41,41],'Failstacks':[0,0,0,0,0]}

{'Reblaths': [41, 41, 41, 41], 'Failstacks': [0, 0, 0, 0, 0]}

In [10]:
selfstate = {'Reblaths':[40,0,0,0],'Failstacks':[20,5,10,99,100]}
testing = [3,4]
print(testing[0])
print(selfstate['Reblaths'][testing[0]]>0)
print(selfstate['Failstacks'][testing[1]])
match testing[0]:
    case 0: basechance = .076923
    case 1: basechance = .062500
    case 2: basechance = .020000
    case 3: basechance = .003000
print(basechance)
enhancementchance = basechance + .1*basechance*selfstate['Failstacks'][testing[1]-1]
print(enhancementchance)
print(selfstate['Failstacks'])
selfstate['Failstacks'].sort()
print(selfstate['Failstacks'])
print(np.random.random(1)[0])

3
False
100
0.003
0.03270000000000001
[20, 5, 10, 99, 100]
[5, 10, 20, 99, 100]
0.47724844271607336


In [24]:
PPO??

In [11]:
# shell
class ReblathEnv(Env):
    def __init__(self):
        self.action_space = MultiDiscrete([4,5])
        self.observation_space = Dict({'Reblaths':MultiDiscrete([41,41,41,41]),'Failstacks':MultiDiscrete([401,401,401,401,401])})
        self.state = {'Reblaths':np.array([40,0,0,0]),'Failstacks':np.array([20,20,20,20,20])}
        self.total_taps = 8192
        self.stacks_hit = 0
        self.starti = 20
        
    def step(self, action):
        reward = 0 # initializes the reward
        # perform the action
        if self.state['Reblaths'][action[0]]>0: # if the number of reblaths in the selected reblath type is greater than 0
            # self.state['Failstacks'][action[1]] # current selected failstack amount
            # self.state['Reblaths'][action[0]] # current selected reblath amount
            # action[0] # current selected reblath tier
            match action[0]: # observes what tier of reblath you're enhancing, determines the base enhancement chance
                case 0: basechance = .076923
                case 1: basechance = .062500
                case 2: basechance = .020000
                case 3: basechance = .003000
            enhancementchance = basechance + .1*basechance*self.state['Failstacks'][action[1]] #determines the enhancement chance
            if np.random.random(1)[0] < enhancementchance: # if the enhancement succeeds
                if action[0]==3: # and tet->pen reblath succeeded
                    self.state['Reblaths'][action[0]] -= 1 # decrease the current amount of reblath in that tier by 1
                    self.state['Reblaths'][0] += 1 #obtain another pri reblath
                    #reward += 70-self.state['Failstacks'][action[1]]
                    reward += enhancementchance*(70-self.state['Failstacks'][action[1]])+(1-enhancementchance)*(3+action[0])
                    self.state['Failstacks'][action[1]] = 70 # set the failstack to 70 (devour)
                else: # and it was any other tier of reblath that succeeded
                    self.state['Reblaths'][action[0]] -= 1 # decrease the current amount of reblath in that tier by 1
                    self.state['Reblaths'][action[0]+1] += 1 # increase the current amount of reblath in next tier by 1
                    #reward += 20-self.state['Failstacks'][action[1]]
                    reward += enhancementchance*(self.starti-self.state['Failstacks'][action[1]])+(1-enhancementchance)*(3+action[0])
                    self.state['Failstacks'][action[1]] = self.starti # set the failstack to the starting stack
            else: # if the enhancement fails
                if action[0]==0: #and it was pri->duo reblath that failed
                    reward += 3 + action[0]
                    self.state['Failstacks'][action[1]] += 3+action[0] # increase failstack by 3+index (3,4,5,6)
                else: # and it was any other tier of reblath that failed
                    self.state['Reblaths'][action[0]] -= 1 # decrease the current amount of reblath in that tier by 1
                    self.state['Reblaths'][action[0]-1] += 1 # increase the current amount of reblath in previous tier by 1
                    reward += 3 + action[0]
                    self.state['Failstacks'][action[1]] += 3+action[0] # increase failstack by 3+index (3,4,5,6)
        else: #if the number of reblaths in the selected reblath type is not greater than 0
            reward -= 90 # cant enhance something that doesnt exist! bad!
            #self.total_taps +=1

        if self.state['Failstacks'][action[1]] >= 110: # if the bot actually made a 110 stack with this action 
            reward += 110 # gives a good reward payout
            self.state['Failstacks'][action[1]] = 20 # sets the failstack value to 20
            self.stacks_hit += 1
        
        self.total_taps -= 1
        if self.total_taps <=0:
            done = True
        else:
            done = False
        self.state['Failstacks'].sort()
        info = {"Stacks Hit":[self.stacks_hit],'Failstack List':[self.state['Failstacks'][0],self.state['Failstacks'][1],self.state['Failstacks'][2],self.state['Failstacks'][3],self.state['Failstacks'][4]],"Reb List":[self.state['Reblaths'][0],self.state['Reblaths'][1],self.state['Reblaths'][2],self.state['Reblaths'][3]]}
        #reward = reward/116 # this is the reward normalization - impliment after experimenting with backpropagation
        return self.state, reward, done, info

    def render(self):
        # nothing happens here
        pass
    def reset(self):
        self.state = {'Reblaths':np.array([40,0,0,0]),'Failstacks':np.array([20,20,20,20,20])}
        self.total_taps = 8192
        self.stacks_hit = 0
        self.state['Failstacks'].sort()
        return self.state
    def testcase(self):
        self.state = {'Reblaths':np.array([0,0,0,40]),'Failstacks':np.array([20,20,20,80,105])}
        self.total_taps = 8192
        self.stacks_hit = 0
        self.state['Failstacks'].sort()
        return self.state

In [12]:
env = ReblathEnv()

In [13]:
check_env(env)

In [14]:
env.observation_space.sample()

OrderedDict([('Failstacks', array([318, 210, 382, 248, 264], dtype=int64)),
             ('Reblaths', array([11,  6,  4,  0], dtype=int64))])

In [15]:
env.testcase()

{'Reblaths': array([ 0,  0,  0, 40]),
 'Failstacks': array([ 20,  20,  20,  80, 105])}

In [16]:
env.step([2,4])

({'Reblaths': array([ 0,  0,  0, 40]),
  'Failstacks': array([ 20,  20,  20,  80, 105])},
 -90,
 False,
 {'Stacks Hit': [0],
  'Failstack List': [20, 20, 20, 80, 105],
  'Reb List': [0, 0, 0, 40]})

In [17]:
env.reset()

In [18]:
episodes = 20
for episode in range(1,episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}  {}'.format(episode,score,info))

Episode:1 Score:-448755.4120406005  {'Stacks Hit': [0], 'Failstack List': [32, 35, 35, 39, 44], 'Reb List': [40, 0, 0, 0]}
Episode:2 Score:-469703.78058280057  {'Stacks Hit': [0], 'Failstack List': [20, 26, 29, 30, 37], 'Reb List': [40, 0, 0, 0]}
Episode:3 Score:-469797.35180260014  {'Stacks Hit': [0], 'Failstack List': [20, 23, 26, 42, 45], 'Reb List': [40, 0, 0, 0]}
Episode:4 Score:-448056.0620184  {'Stacks Hit': [0], 'Failstack List': [20, 24, 37, 43, 45], 'Reb List': [40, 0, 0, 0]}
Episode:5 Score:-458112.79906420014  {'Stacks Hit': [0], 'Failstack List': [20, 20, 23, 29, 44], 'Reb List': [39, 1, 0, 0]}
Episode:6 Score:-459493.3647438001  {'Stacks Hit': [0], 'Failstack List': [20, 27, 29, 31, 39], 'Reb List': [40, 0, 0, 0]}
Episode:7 Score:-460323.7080114001  {'Stacks Hit': [0], 'Failstack List': [26, 27, 30, 35, 36], 'Reb List': [40, 0, 0, 0]}
Episode:8 Score:-462255.1531998006  {'Stacks Hit': [0], 'Failstack List': [20, 20, 27, 28, 39], 'Reb List': [39, 1, 0, 0]}
Episode:9 Score:

In [19]:
log_path = os.path.join('training','logs')
model = PPO('MultiInputPolicy',env, verbose=0,tensorboard_log=log_path,learning_rate=.0005)

In [20]:
env.reset()
model.learn(total_timesteps=10000)

In [21]:
reblath_path = os.path.join('training','models','reblath_PPO_average_6milly')
model.save(reblath_path)

In [129]:
del model

In [130]:
model = PPO.load(reblath_path,env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
episodes = 10
for episode in range(1,episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        action = model.predict(obs)[0] # now using model here
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} {}'.format(episode,score,info))
env.reset()

Episode:1 Score:21314.572847199754 {'Stacks Hit': [10], 'Failstack List': [28, 52, 53, 53, 94], 'Reb List': [40, 0, 0, 0]}
Episode:2 Score:20843.186666599813 {'Stacks Hit': [8], 'Failstack List': [26, 26, 27, 27, 65], 'Reb List': [39, 1, 0, 0]}
Episode:3 Score:19973.827246199777 {'Stacks Hit': [1], 'Failstack List': [23, 23, 48, 49, 66], 'Reb List': [40, 0, 0, 0]}
Episode:4 Score:20410.193979199837 {'Stacks Hit': [5], 'Failstack List': [28, 31, 39, 42, 76], 'Reb List': [38, 2, 0, 0]}
Episode:5 Score:21135.747265599854 {'Stacks Hit': [10], 'Failstack List': [27, 27, 43, 48, 79], 'Reb List': [39, 1, 0, 0]}
Episode:6 Score:10905.405001199882 {'Stacks Hit': [4], 'Failstack List': [26, 27, 29, 48, 80], 'Reb List': [40, 0, 0, 0]}
Episode:7 Score:3105.6774411999886 {'Stacks Hit': [8], 'Failstack List': [20, 28, 30, 42, 73], 'Reb List': [39, 1, 0, 0]}
Episode:8 Score:20599.114819199815 {'Stacks Hit': [5], 'Failstack List': [20, 20, 23, 26, 46], 'Reb List': [38, 1, 0, 1]}
Episode:9 Score:20921.

{'Reblaths': array([40,  0,  0,  0]),
 'Failstacks': array([20, 20, 20, 20, 20])}

In [30]:
env.reset()

{'Reblaths': [40, 0, 0, 0], 'Failstacks': [20, 20, 20, 20, 20]}

In [23]:
evaluate_policy(model,env,n_eval_episodes=5)

C:\Users\austi\.virtualenvs\pyml-FJ61xwZS\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(20947.183492703924, 348.02318911105044)